In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, IntegerType
import pandas as pd
from IPython.display import display, clear_output

# change for compatibility between Spark and Kafka
spark = SparkSession.builder \
    .appName("KafkaStreamingExample") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2") \
    .getOrCreate()

print("before stream...")

# Kafka connection details
bootstrap_servers = "pkc-56d1g.eastus.azure.confluent.cloud:9092"
kafka_topic = "Saeed_topic"
kafka_username = "JUKQQM4ZM632RECA"
kafka_password = "UUkrPuSttgOC0U9lY3ZansNsKfN9fbxZPFwrGxudDrfv+knTD4rCwK+KdIzVPX0D"

# Define schema for the incoming JSON data
schema = StructType() \
    .add("eventType", StringType()) \
    .add("customerId", StringType()) \
    .add("productId", StringType()) \
    .add("timestamp", StringType()) \
    .add("metadata", StructType()
         .add("category", StringType(), True)
         .add("source", StringType(), True)) \
    .add("quantity", IntegerType(), True) \
    .add("totalAmount", StringType(), True) \
    .add("paymentMethod", StringType(), True) \
    .add("recommendedProductId", StringType(), True) \
    .add("algorithm", StringType(), True)

# Read data from Kafka topic as a streaming DataFrame
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .option("kafka.security.protocol", "SASL_SSL") \
    .option("kafka.sasl.mechanism", "PLAIN") \
    .option("kafka.sasl.jaas.config",
            f'org.apache.kafka.common.security.plain.PlainLoginModule required username="{kafka_username}" password="{kafka_password}";') \
    .load()

json_df = df.selectExpr("CAST(value AS STRING)").select(from_json("value", schema).alias("data")).select("data.*")


# Define HDFS path
hdfs_path = "/stream"

# Write the result to HDFS in Parquet format
query = json_df \
    .writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", hdfs_path) \
    .option("checkpointLocation", "/check_point") \
    .start()


print("Starting stream...")

query.awaitTermination()


In [ ]:
spark.stop()